In [ ]:
# ! pip install pyDOE
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import grad
from pyDOE import lhs
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import pickle

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

torch.manual_seed(1234)
np.random.seed(1234)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(1234)

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(device)

In [4]:
# Creating the geometry

x_min = 0.0
x_max = 0.2
y_min = 0.0
y_max = 0.2
ub = np.array([x_max, y_max])
lb = np.array([x_min, y_min])

N_b = 100
N_w = 100
N_c = 20000

def getData():

    wall1_xy = [x_min, y_min] + [0.0, y_max] * lhs(2, N_b)
    wall1_T = np.zeros((N_b, 1))

    wall2_xy = [x_max, y_min] + [0.0, y_max] * lhs(2, N_b)
    wall2_T = np.zeros((N_b, 1))

    wall3_xy = [x_min, y_max] + [x_max, 0.0] * lhs(2, N_w)
    wall3_T = np.ones((N_b, 1))

    wall4_xy = [x_min, y_min] + [x_max, 0.0] * lhs(2, N_w)
    wall4_T = np.zeros((N_b, 1))

    xy_bnd = np.concatenate([wall1_xy, wall2_xy, wall3_xy, wall4_xy], axis=0)
    T_bnd = np.concatenate([wall1_T, wall2_T, wall3_T, wall4_T], axis=0)

    xy_col = lb + (ub - lb) * lhs(2, N_c)
    xy_col = np.concatenate((xy_col, xy_bnd), axis=0)

    xy_bnd = torch.tensor(xy_bnd, dtype=torch.float32).to(device)
    T_bnd = torch.tensor(T_bnd, dtype=torch.float32).to(device)
    xy_col = torch.tensor(xy_col, dtype=torch.float32).to(device)

    return xy_col, xy_bnd, T_bnd

xy_col, xy_bnd, T_bnd = getData()

In [ ]:
# Learning process

def plotLoss(losses_dict, path, info=["B.C.", "P.D.E."]):
    fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(10, 6))
    axes[0].set_yscale("log")
    for i, j in zip(range(2), info):
        axes[i].plot(losses_dict[j.lower()])
        axes[i].set_title(j)
    plt.show()
    fig.savefig(path)

def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.zeros_(m.bias.data)

class layer(nn.Module):

    def __init__(self, n_in, n_out, activation):
        super().__init__()
        self.layer = nn.Linear(n_in, n_out)
        self.activation = activation

    def forward(self, x):
        x = self.layer(x)
        if self.activation:
            x = self.activation(x)
        return x

class DNN(nn.Module):

    def __init__(self, dim_in, dim_out, n_layer, n_node, ub, lb, activation=nn.Tanh()):
        super().__init__()
        self.net = nn.ModuleList()
        self.net.append(layer(dim_in, n_node, activation))
        for _ in range(n_layer):
            self.net.append(layer(n_node, n_node, activation))
        self.net.append(layer(n_node, dim_out, activation=None))
        self.ub = torch.tensor(ub, dtype=torch.float).to(device)
        self.lb = torch.tensor(lb, dtype=torch.float).to(device)
        self.net.apply(weights_init)

    def forward(self, x):
        out = x
        for layer in self.net:
            out = layer(out)
        return out

class PINN:

    def __init__(self) -> None:
        self.net = DNN(dim_in=2, dim_out=1, n_layer=4, n_node=50, ub=ub, lb=lb).to(
            device
        )

        self.lbfgs = torch.optim.LBFGS(
            self.net.parameters(),
            lr=0.1,
            max_iter=20000,
            max_eval=20000,
            tolerance_grad=1e-5,
            tolerance_change= 1 * np.finfo(float).eps,
            history_size=50,
            line_search_fn="strong_wolfe",
        )

        self.adam = torch.optim.Adam(self.net.parameters(), lr=5e-4)
        self.losses = {"bc": [], "pde": []}
        self.iter = 0

    def predict(self, xy):
        out = self.net(xy)

        T = out

        return T

    def bc_loss(self, xy):
        T = self.predict(xy)

        mse_bc = torch.mean(torch.square(T - T_bnd))

        return mse_bc


    def pde_loss(self, xy):
        xy = xy.clone()
        xy.requires_grad = True

        T = self.predict(xy)

        T_out = grad(T.sum(), xy, create_graph=True)[0]
        T_out2 = grad(T_out.sum(), xy, create_graph=True)[0]

        T_x = T_out[:, 0:1]
        T_y = T_out[:, 1:2]
        T_xx = T_out2[:, 0:1]
        T_yy = T_out2[:, 1:2]

        f0 = T_xx + T_yy

        mse_f0 = torch.mean(torch.square(f0))

        mse_pde = mse_f0

        return mse_pde

    def closure(self):

        self.lbfgs.zero_grad()
        self.adam.zero_grad()

        mse_bc = self.bc_loss(xy_bnd)
        mse_pde = self.pde_loss(xy_col)

        loss = mse_bc + mse_pde

        loss.backward()

        self.losses["bc"].append(mse_bc.detach().cpu().item())
        self.losses["pde"].append(mse_pde.detach().cpu().item())

        self.iter += 1

        print(
            f"\r It: {self.iter} Loss: {loss.item():.5e} BC: {mse_bc.item():.3e}  pde: {mse_pde.item():.3e}",
            end="",
        )

        if self.iter % 100 == 0:
            print("")

        return loss

if __name__ == "__main__":

    pinn = PINN()
    start_time = time.time()

    for i in range(4000):
        pinn.closure()
        pinn.adam.step()
    pinn.lbfgs.step(pinn.closure)

    print("--- %s seconds ---" % (time.time() - start_time))
    print(f'-- {(time.time() - start_time)/60} mins --')
    torch.save(pinn.net.state_dict(), "/content/Param.pt")
    plotLoss(pinn.losses, "/content/LossCurve.png", ["BC", "PDE"])


In [ ]:
# Visualizing results

pinn = PINN()

pinn.net.load_state_dict(torch.load("/content/Param.pt"))

x = np.arange(x_min, x_max, 0.001)
y = np.arange(y_min, y_max, 0.001)

X, Y = np.meshgrid(x, y)

x = X.reshape(-1, 1)
y = Y.reshape(-1, 1)

xy = np.concatenate([x, y], axis=1)
xy = torch.tensor(xy, dtype=torch.float32).to(device)

with torch.no_grad():
    u = pinn.predict(xy)
    u = u.cpu().numpy()
    u = u.reshape(Y.shape)

fig, axes = plt.subplots(1, 1, figsize=(11, 12), sharex=True)
data = (u)
labels = ["$u(x,y)$"]
for i in range(1):
    ax = axes
    im = ax.imshow(
        data, cmap="rainbow", extent=[x_min, x_max, y_min, y_max], origin="lower"
    )
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad="3%")
    fig.colorbar(im, cax=cax, label=labels[i])
    ax.set_title(labels[i])
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_aspect("equal")

fig.tight_layout()

fig.savefig("/content/Sol.png", dpi=500)